In [58]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import log_loss
from scipy.spatial import distance


In [53]:
# PREDICTION VARIABLES
num_random_seeds = 5
num_perturbed_models = 50

test_size = 0.33
validation_size = 0.20

EPSILON = 0.1

TARGET_VARIABLE = "qualified_gagne_3"
prediction_output = "predictions/obermeyer/linear_wp_"+TARGET_VARIABLE+".csv"

In [3]:
data_source = "data/obermeyer/obermeyer_data_cleaned.csv"
features = ['dem_female', 'dem_age_band_18-24_tm1', 'dem_age_band_25-34_tm1', 'dem_age_band_35-44_tm1', 'dem_age_band_45-54_tm1',
            'dem_age_band_55-64_tm1', 'dem_age_band_65-74_tm1', 'dem_age_band_75+_tm1', 'hypertension_elixhauser_tm1', 'cost_dialysis_tm1',
            'cost_emergency_tm1', 'cost_home_health_tm1', 'cost_ip_medical_tm1', 'cost_ip_surgical_tm1', 'cost_laboratory_tm1',
            'cost_op_primary_care_tm1', 'cost_op_specialists_tm1', 'cost_op_surgery_tm1', 'cost_other_tm1', 'cost_pharmacy_tm1',
            'cost_physical_therapy_tm1', 'cost_radiology_tm1', 'gagne_sum_tm1']
other_variables = ['person_id', 'gagne_sum_t', 'cost_t']

In [4]:
df = pd.read_csv(data_source)
X = df[features+other_variables]
y = df[TARGET_VARIABLE]

In [5]:
def get_baseline_model_coefficients(X_train, y_train):
    model = LinearRegression()
    model.fit(X_train, y_train)
    return model.coef_

In [61]:
def get_perturbed_weights(baseline_weights, epsilon):
    random_vector = np.random.randn(*baseline_weights.shape)
    random_vector = random_vector / np.linalg.norm(random_vector)
    perturbation = np.random.uniform(0, epsilon) * random_vector
    w = baseline_weights + perturbation
    if distance.euclidean(w, baseline_weights) > epsilon:
        print(f"L2 norm {distance.euclidean(w, baseline_weights)} more than {epsilon} away")
    return w

In [7]:
def get_predictions_columns(X_test, baseline_weights, epsilon):
    predictions = []
    columns = []
    for i in tqdm(range(num_perturbed_models)):
        perturbed_weights = get_perturbed_weights(baseline_weights, epsilon)
        predictions.append(np.dot(X_test, perturbed_weights))
        columns.append(f'm_{i+1}')
    return predictions, columns

In [63]:
def rss_loss(y_true, y_pred):
    return ((y_true - y_pred)** 2).sum()

In [ ]:
output = []
num_weights = len(features) + 1
for random_seed in range(num_random_seeds):
    print("random seed", random_seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_seed)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation_size, random_state=random_seed)

    cost = X_test["cost_t"].to_numpy()
    gagne = X_test["gagne_sum_t"].to_numpy()
    person_id = X_test['person_id'].to_numpy()

    X_train = X_train.drop(columns=other_variables).to_numpy()
    y_train = y_train.to_numpy()
    X_val = X_val.drop(columns=other_variables).to_numpy()
    y_val = y_val.to_numpy()
    X_test = X_test.drop(columns=other_variables).to_numpy()
    y_test = y_test.to_numpy()

    # Orthonormalize the training matrix
    intercept_idx = 0
    X_train = np.insert(X_train, intercept_idx, 1.0, axis=1)
    q, r = sp.linalg.qr(X_train) # q is the orthonormalized basis, r is the upper triangular
    q = q[:,:num_weights] # keep only columns of q that actually get multiplied
    r = r[:num_weights] # make r a square matrix

    r_inverse = sp.linalg.inv(r) # q = q * r * r' = X * r'
    X_test = np.insert(X_test, intercept_idx, 1.0, axis=1)
    X_ortho_test = X_test @ r_inverse
    X_val = np.insert(X_val, intercept_idx, 1.0, axis=1)
    X_ortho_val = X_val @ r_inverse

    predictions = {}
    training_loss = {}
    validation_loss = {}
    baseline_weights = get_baseline_model_coefficients(q, y_train)
    baseline_loss = rss_loss(y_train, np.dot(q, baseline_weights))
    
    print(f"baseline training_loss {baseline_loss}")
    count = 0
    for i in tqdm(range(num_perturbed_models)):
        perturbed_weights = get_perturbed_weights(baseline_weights, EPSILON)
        predictions[f'm_{i+1}'] = np.dot(X_ortho_test, perturbed_weights)
        t_loss = rss_loss(y_train, np.dot(q, perturbed_weights))
        if t_loss > baseline_loss + EPSILON:
            count += 1
            # print(f"oh noooooooooooooo {t_loss} too big")
        training_loss[f'm_{i+1}'] = log_loss(y_train, np.dot(q, perturbed_weights))
        validation_loss[f'm_{i+1}'] = log_loss(y_val, np.dot(X_ortho_val, perturbed_weights))

    print(f"too much error {count} models")
                            
    predictions_df = pd.concat([
        pd.DataFrame(predictions),
        pd.DataFrame(training_loss, index=[0]),
        pd.DataFrame(validation_loss, index=[0])]).reset_index(drop=True)
    
    predictions_df["y"] = np.concatenate([y_test, [np.nan, np.nan]])
    predictions_df["person_id"] = np.concatenate([person_id, [-2, -1]]) # -1 indicates validation loss, -2 indicates training loss
    predictions_df['cost_t'] = np.concatenate([cost, [np.nan, np.nan]]) 
    predictions_df['gagne_sum_t'] = np.concatenate([gagne, [np.nan, np.nan]])
    predictions_df["seed"] = random_seed

    output.append(predictions_df)
    
output = pd.concat(output)

random seed 0
baseline training_loss 2981.304928434374


100%|██████████| 100/100 [00:00<00:00, 146.87it/s]


too much error 42 models
random seed 1
baseline training_loss 2954.6509778635705


100%|██████████| 100/100 [00:00<00:00, 144.89it/s]


too much error 37 models
random seed 2
baseline training_loss 2935.418350904594


...

too much error 39 models
random seed 3
baseline training_loss 2999.6081504893073


100%|██████████| 100/100 [00:01<00:00, 98.87it/s]


too much error 42 models
random seed 4
baseline training_loss 3019.967853428894


100%|██████████| 100/100 [00:00<00:00, 102.19it/s]


too much error 37 models


In [47]:
output.head()

,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9,m_10,y,person_id,cost_t,gagne_sum_t,seed
0,-0.019835,-0.020067,-0.020386,-0.019915,-0.020128,-0.020007,-0.019889,-0.019885,-0.019995,-0.019854,0.0,2545,0.009628,1.0,0
1,0.080933,0.080950,0.080478,0.080819,0.080785,0.080616,0.080869,0.080984,0.080887,0.080838,0.0,8198,0.004905,2.0,0
2,-0.072258,-0.072819,-0.073480,-0.072972,-0.072733,-0.073173,-0.072908,-0.073571,-0.073039,-0.072832,0.0,46461,0.009446,0.0,0
3,-0.208993,-0.208892,-0.207873,-0.208485,-0.208466,-0.208490,-0.208448,-0.208626,-0.208503,-0.208484,0.0,30620,0.002361,0.0,0
4,0.107347,0.106343,0.107324,0.107374,0.107178,0.107556,0.107395,0.107287,0.107387,0.106992,0.0,47418,0.003996,2.0,0


In [30]:
output.to_csv(prediction_output, index=False)

In [50]:
for i in range(10):
    se = np.mean((output['m_'+str(i+1)] - output['y']) ** 2)
    print(se)

0.11507308429942796
0.11507999056767682
0.1150663174449882
0.11509745879770553
0.11506694099084161
0.11505165482684833
0.11508433979567395
0.11507781670692932
0.115089960599257
0.11505346600623309
